In [2]:
import time
import multiprocessing

# Multiprocessing module

The **`multiprocessing`** module allows us to open different threads and do independent computations (at the same time) in different cpu cores.

- **`multiprocessing.Process`**: creates a new process identifier. This can be used to start a task that runs as an independent child process in the operating sstem. 

- **`multiprocessing.Pool`**: creates a pool of workkers that share a chunk of work and return an agretated result.

- **`multiprocessing.Queue`**: A FIFO queue allowing multiple producers and consumers.

- **`multiprocessing.Pipe`**: A uni- or bidirectional communication channel between two processes.

- **`multiprocessing.Manager`**: A high-level managed interface to share Python objects between processes.


### Some interesting urls
- https://www.youtube.com/watch?v=s1SkCYMnfbY
- https://pymotw.com/2/multiprocessing/basics.html
- https://www.youtube.com/watch?v=s1SkCYMnfbY
- http://joshuagoings.com/2015/08/31/embarassingly-parallel-tasks-in-python/
- http://chriskiehl.com/article/parallelism-in-one-line/
- https://www.ibm.com/developerworks/aix/library/au-threadingpython/

#### BLAS and multiprocessing

- https://github.com/obspy/obspy/wiki/Notes-on-Parallel-Processing-with-Python-and-ObsPy

#### Joblib
- http://www.admin-magazine.com/HPC/Articles/Parallel-Python-with-Joblib

## Setting number of threads

Setting the number of threads for a given program execution
- https://stackoverflow.com/questions/39381974/how-to-set-the-max-thread-a-python-script-could-use-when-calling-from-shell

Setting the number of threads for openBLAS/MKL
- https://stackoverflow.com/questions/19257070/unintented-multithreading-in-python-scikit-learn

Some solutions

    python some_program.py --nthread=2

In [3]:
multiprocessing.Process

multiprocessing.context.Process

In [4]:
multiprocessing.Manager

<bound method BaseContext.Manager of <multiprocessing.context.DefaultContext object at 0x7f15eae3c358>>

### Example 1: multiprocessing.pool

Pool object which offers a convenient means of parallelizing the execution of a function across multiple input values, distributing the input data across processes (data parallelism). 
 
Easy way to split a computation that is embarrasinly parallel on a set of parallel processes.

    pool = multiprocessing.Pool(processes=n_parallel_processess)
    pool.map(some_function, arguments_for_some_function)

Important: You cannot use pool.map inside the interpreter. You need to run it as a script inside `if __name__ == '__main__'`




In [5]:
import numpy
import numpy.random as random

In [6]:
def calculate_pi(nbr_estimates):
    steps = range(int(nbr_estimates))
    nbr_trials_in_unit_circle = 0
    for step in steps:
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        is_in_unit_circle = x * x + y * y <= 1.0
        nbr_trials_in_unit_circle += is_in_unit_circle
    return nbr_trials_in_unit_circle

In [7]:
def estimate_nbr_points_in_quarter_circle(nbr_estimates):
    nbr_trials_in_quarter_unit_circle = 0
    for step in range(int(nbr_estimates)):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        is_in_unit_circle = x * x + y * y <= 1.0
        nbr_trials_in_quarter_unit_circle += is_in_unit_circle
        
    return nbr_trials_in_quarter_unit_circle

In [8]:
nbr_samples_in_total = 1e5
nbr_parallel_blocks = 4
pool = multiprocessing.Pool(processes=nbr_parallel_blocks)
nbr_samples_per_worker = nbr_samples_in_total / nbr_parallel_blocks
print("Making {} samples per worker".format(nbr_samples_per_worker))
nbr_trials_per_process = [nbr_samples_per_worker] * nbr_parallel_blocks
t1 = time.time()
nbr_in_unit_circles = pool.map(calculate_pi, nbr_trials_per_process)
pi_estimate = sum(nbr_in_unit_circles) * 4 / nbr_samples_in_total
print("Estimated pi", pi_estimate)
print("Delta:", time.time() - t1)

Making 25000.0 samples per worker
Estimated pi 3.14064
Delta: 0.16458439826965332


### Example 2: Multiprocess.Process

We can create a python process doing  `multiprocessing.Process`. 

        p = multiprocessing.Process(target=function, args=tuple_args_for_function)
        p.start()
        # Good practize to do
        p.join()
        
 
 
run()

    Method representing the process’s activity.

    You may override this method in a subclass. The standard run() method invokes the callable object passed to the object’s constructor as the target argument, if any, with sequential and keyword arguments taken from the args and kwargs arguments, respectively.



In [24]:
import time
from time import sleep

In [49]:
import multiprocessing

def worker(n):
    """worker function"""
    for i in range(3):
        sleep(2)
        print('Worker ', n)
    return

if __name__ == '__main__':
    jobs = []
    for i in range(10):
        p = multiprocessing.Process(target=worker, args=(i, ))
        jobs.append(p)
        p.start()

Worker  0
Worker  1
Worker  2
Worker  3
Worker  4
Worker  5
Worker  6
Worker  8
Worker  7
Worker  9
Worker  2
Worker  0
Worker  1
Worker  3
Worker  4
Worker  5
Worker  6
Worker  7
Worker  8
Worker  9
Worker  2
Worker  1
Worker  0
Worker  3
Worker  4
Worker  6
Worker  5
Worker  7
Worker  8
Worker  9


### Example 3

In [45]:
import multiprocessing
import time

def worker():
    name = multiprocessing.current_process().name
    print (name, 'Starting')
    time.sleep(2)
    print (name, 'Exiting')

def my_service():
    name = multiprocessing.current_process().name
    print (name, 'Starting')
    time.sleep(3)
    print (name, 'Exiting')

service  = multiprocessing.Process(name='my_service', target=my_service)
worker_1 = multiprocessing.Process(name='worker 1', target=worker)
worker_2 = multiprocessing.Process(target=worker) # use default name

worker_1.start()
worker_2.start()
service.start()

worker 1 Starting
my_service Starting
Process-110 Starting
worker 1 Exiting
Process-110 Exiting
my_service Exiting


### Example 3

In [9]:
square_result = []

def calc_square(numbers):
    global squared_result
    for n in numbers:
        square_result.append(n*n)

In [10]:
arr = [2 for i in range(10**5)]

In [11]:
p1 = multiprocessing.Process(target = calc_square, args= (arr,) ) 

In [12]:
p1.start()
p1.join()

In [13]:
square_result

[]

Why is squared result empty?

The reason is because the new process has its own adress space (virtual memory). Therefore program variables are not shared between two processes. Interprocess communication (IPC) is needed to share data between two processes.


### Example 3: Share data between processes


- multiprocessing.Array
- multiprocessing.Value



In [14]:
def calc_square(numbers, result):
    # this does not work
    #for n in numbers:
    #    result.append(n*n) 
    for idx, n in enumerate(numbers):
        result[idx] = n*n

numbers = [2,3,4,5,6,7]
result = multiprocessing.Array("i", len(numbers))

In [15]:
result[:]

[0, 0, 0, 0, 0, 0]

In [16]:
p = multiprocessing.Process(target=calc_square, args=(numbers, result))
p.start()
p.join()

In [17]:
result[:]

[4, 9, 16, 25, 36, 49]

## Defining and executing processes

http://sebastianraschka.com/Articles/2014_multiprocessing.html